# Langchain Basics

## What you will learn in this course? 🧐🧐

Let's dive deeper into how Langchain actually works. In this course, we will cover:

* Langchain client 
* Langchain API 
* Prompt Templating
* Models
* Langchain Expression Language - LCEL

## Basic architecture of a Langchain Application 

When you are building a langchain application, you need to visualize how it will look like in production. Your application will be structured with the following components:

* **An API server** 👉 This is the skeleton of your application where you'll write all the logic of your application
* **Third Party Tools** 👉 Most likely your API will need to use an externally hosted LLM (like OpenAI ChatGPT, Mistral or even your own custom trained model hosted on a separate server), or use other external tools like Google Search, Scrapers etc.
* **Clients** 👉 Then finally, the way the end-user will use your API is through a Web application (like streamlit, Gradio etc.) or directly with Jupyter Notebooks

![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Langchain-application-architecture.png)

For the demo below, here is what we are going to build:

* API 👉 This will be a very basic Chatbot
* Third Party 👉 We will use Mistral as our external pre-trained model 
* Clients 👉 We will show you how to consume the API on a Jupyter notebook

## Requirements

* Create an account in Mistral and get an API key. Here is how to do it 👇

<Video video="https://vimeo.com/1017970631"/>

## API - Full code

Let's first start by presenting the full application for you to see how a langchain application would look like. We'll then dive into each part of the code step by step. This application is based [on this great Langchain tutorial](https://python.langchain.com/docs/tutorials/llm_chain/)


Here is the source code of the demo:

* [Langchain Demo App](https://github.com/JedhaBootcamp/Sample_Langchain_app)

Feel free to:

```bash 
git clone https://github.com/JedhaBootcamp/Sample_Langchain_app.git
```

Now you can run the application simply by running:

```bash
docker run -p 7860:7860 -e MISTRAL_API_KEY=YOUR_MISTRAL_API_KEY jedha/langchain-base
```

It will start a web server that you can access at:

* **http://localhost:7860/chain/playground**


Now this app is pretty basic but it's already a great start for us to understand the API. Let's dive into each part of the code to deepen our understanding. 


<Note type="important">

For the rest of the demo, we will be using this API in production on HuggingFace at the URL:

* `https://antoinekrajnc-sample-langchain-api.hf.space/`

</Note>


## Additional client requirements 

When we'll be using the API, the **client (like a jupyter notebook)** will need to have the following dependencies installed:

* `langchain`
* `langchain-community `
* `langchain-mistralai`
* `langchain-openai`
* `langserve[all]`
* `langgraph`
* `fastapi[standard]`

<Note type="tip">

If you do not want to install all these packages on your machine, you should run a docker container. I advise you to use `jupyter/datascience-notebook`.

```bash 
docker run -v $(pwd):/home/jovyan -p 8888:8888 jupyter/datascience-notebook
```

This will automatically run jupyter lab at `http://localhost:8888`. You can then either open up your web browser or you can also use the Jupyter kernel in VSCode. Here is how:

* [Run Jupyter Notebook container in VSCode](https://medium.com/@FredAsDev/connect-vs-code-jupyter-notebook-to-a-jupyter-container-a63293f29325)

</Note>


<Note type="important">

This is for the **Client** setup. Even though we will have to install similar dependencies for the API, I just want to make sure that there are two independent elements: 

* A client (this jupyter notebook)
* A server (the API - that is hosted on HuggingFace)

</Note>


## Client-side: Interact with the API

In [1]:
# If you don't have the dependencies already installed
!pip install langchain -q
!pip install langchain_mistralai -q
!pip install langserve -q
!pip install fastapi -q

<Note type="important">

The API server is a very small one to test **very small requests**, don't try to use it for large contexts etc, it will just break the server.

</Note>


To interact with a LangServe API clients can use it pretty easily using `RemoteRunnable` class

In [2]:
from langserve import RemoteRunnable

# This is the production API from the demo github repo - https://github.com/JedhaBootcamp/Sample_Langchain_app
HOST = "https://antoinekrajnc-sample-langchain-api.hf.space"
ENDPOINT="/chain/"

translator = RemoteRunnable(f"{HOST}{ENDPOINT}")
translator.invoke({"language": "French", "text": "What is the name of the most famous Star Wars bounty hunter?"})

KeyboardInterrupt: 

In [2]:
## YOU CAN ALSO USE THE CODE BELOW TO PRINT TOKEN BY TOKEN (LIKE IN CHATGPT APP)
for token in translator.stream({"language": "French", "text": "What is the name of the most Star Wars bounty hunter?"}):
    print(token, end='', flush=True)

The most famous bounty hunter from Star Wars is Boba Fett.

Translated into French, it would be:

"Quel est le nom du chasseur de primes Star Wars le plus célèbre?"

And the answer:

"Boba Fett" (the name remains the same in French).

Alright now you know **how to interact from a client to the API using Langchain**. Pretty easy right? For the next section, we'll dive deeper into each concept behind the API so that you can understand the code better. 

<Note type="important">

If you want to follow along, keep your client Jupyter Notebook up so that you can run the commands below

</Note>

## API-Side: Deep dive into the code

### Prompt Template

Let's start by talking about prompts and prompt templates. When calling an LLM, it is likely that you have some kind of instructions that you want it to follow. In the above demo:

```python
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])
```

We want the LLM to translate any sentence in a given `{language}`. 

Now there are three types of Prompt Templates:

#### Prompt Templates Overview

| **Type**                | **Description**                                       |
|-------------------------|-------------------------------------------------------|
| **[`PromptTemplate`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html)** | Formats a single string for simpler inputs.           |
| **[`ChatPromptTemplate`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)**   | Formats a list of messages for complex interactions.   |
| **[`MessagesPlaceholder`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)**  | Inserts a list of messages at a specific point.        |


Let's see an example with each of them to see how they work. 

#### `PromptTemplate`

`PromptTemplate` is the most basic class that is usually used when you have one simple prompt (like just a single string) to use with a very few configurations: 

<Note type="important">

Note that this class only builds the prompt and does not compute an anwser for the prompt.

</Note>


In [1]:
from langchain_core.prompts import PromptTemplate

message = "Tell me everything you know about this Star Wars Character: {character}"
prompt_template = PromptTemplate.from_template(message) # This is a very simple string with {character} as configurable paramater. 
result = prompt_template.invoke({"character": "Sifo-Dyas"}) # Here we use the `.invoke()` method to actually execute the prompt chain.

result.to_string() # Print just the string result

'Tell me everything you know about this Star Wars Character: Sifo-Dyas'

<Note type="tip">

If that helps, `PromptTemplate` works a bit like f-strings:

* f-string: 

```python
name = "Jedha"
print(f"hello {name}")
```

* PromptTemplate:

```python 
prompt = PromptTemplate.from_template("Hello {name}")
prompt.invoke({"name":"Jedha"})
```

</Note>


#### `ChatPromptTemplate`

`ChatPromptTemplate` handles more complex prompt templates where you can input a sequence of messages with different roles. Let's see how that works:

In [2]:
from langchain_core.prompts import ChatPromptTemplate

# Step 1: Define the interaction template between a human and a droid
template = ChatPromptTemplate.from_messages([
    ("system", "You are a protocol droid designed for assisting sentient beings. Your designation is {name}."),
    ("user", "Greetings, droid. Status report?"),
    ("assistant", "All systems are fully operational, and I am ready to assist you."),
    ("user", "{user_input}"),
])

# Step 2: Fill in the droid's name and user input
prompt_value = template.invoke(
    {
        "name": "R-3PO",  # Name of the droid
        "user_input": "What is your primary function?"  # User asking a question
    }
)

# Step 3: Convert the prompt to messages to simulate the conversation
print(prompt_value.to_messages())

[SystemMessage(content='You are a protocol droid designed for assisting sentient beings. Your designation is R-3PO.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Greetings, droid. Status report?', additional_kwargs={}, response_metadata={}), AIMessage(content='All systems are fully operational, and I am ready to assist you.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your primary function?', additional_kwargs={}, response_metadata={})]


There are three default roles in a `ChatPromptTemplate`:

| Role | Description |
| ---- | ----------- |
| `system` | This is configuration prompt | 
| `user` | This is a human written message | 
| `assistant` | This is the AI response | 


You can definitely change them and add custom roles but we advise you to leave it that way as most models understand it pretty well. Actually in Langchain, you can even setup this messages like this:


In [3]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# system role
SystemMessage(content="You are an astrophysicists AI and know everything space-related")
# user role 
HumanMessage(content="Hi, I want to know about the universe")
# assistant role
AIMessage(content="What a vast and interesting question, is there anything you would like to explore in particular?")

AIMessage(content='What a vast and interesting question, is there anything you would like to explore in particular?', additional_kwargs={}, response_metadata={})

<Note type="note">

There are other types of messages that you can check directly here:

* [Messages](https://python.langchain.com/api_reference/core/messages.html)

</Note>

#### `MessagesPlaceholder`

`MessagesPlaceholder` is a wrapper that will contain a list of different messages with different roles:

In [4]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

# Step 1: Create a placeholder for message history
prompt = MessagesPlaceholder("history", optional=True)  # Leave optional=True in case there are no previous messages.

# Step 2: Add some initial messages to simulate past interaction
prompt.format_messages(
    history=[
        ("system", "You are a protocol droid designed to assist sentient beings."),
        ("human", "Greetings, droid."),
    ]
)

[SystemMessage(content='You are a protocol droid designed to assist sentient beings.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Greetings, droid.', additional_kwargs={}, response_metadata={})]

Now `MessagesPlaceholder` is often used in combination with `ChatPromptTemplate` like this:

In [5]:

# Step 3: Create a new ChatPromptTemplate with MessagesPlaceholder for the conversation history
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a protocol droid with the designation R-3PO."),
        MessagesPlaceholder("history"),  # Placeholder for message history
        ("human", "{question}")  # The human asks a new question
    ]
)

# Step 4: Invoke the prompt with message history and the new question
response = prompt.invoke(
   {
       "history": [("human", "Calculate the coordinates for the jump to lightspeed."), 
                   ("ai", "The jump coordinates are calculated: 12.345, -45.678.")],
       "question": "Now, plot a course to the nearest star system."
   }
)

# Step 5: Simulate the conversation output
print(response.to_messages())

[SystemMessage(content='You are a protocol droid with the designation R-3PO.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Calculate the coordinates for the jump to lightspeed.', additional_kwargs={}, response_metadata={}), AIMessage(content='The jump coordinates are calculated: 12.345, -45.678.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Now, plot a course to the nearest star system.', additional_kwargs={}, response_metadata={})]


This way you can have a dynamic size of messages which is one of the **founding block for adding memory in all LLMs** which we will cover later on 😉


<Note type="note" title="other types of prompt template">

There are other types of Prompt Templates that you can check out here:

* [Prompts](https://python.langchain.com/api_reference/core/prompts.html)


</Note>

### Models 

Alright, let's talk models now. With Langchain, you can use any model you want:

* Proprietary pretrained models like
    * ChatGPT
    * Llama 
    * Mistral
    * ...

* Custom models you trained yourself


We won't cover custom models just yet but we will show you how to use proprieraty models. Langchain built a wrapper around all the major models that you can use for your application. Here is the full list that you can check: 

* [All integrated ChatModels](https://python.langchain.com/docs/integrations/chat/)

To use them, all you need to do is follow this template:

1. Create an account on the given provider (i.e OpenAI, Mistral, AWS)
2. Get an API Key from the provider 
3. Import the model in Langchain

You already have seen an example with Mistral but here is another one with OpenAI:

```bash
export OPENAI_API_KEY="YOUR_OPENAI_API_KEY"
```

Then

```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)
```

### LCEL: Chaining concept with Langchain

This is probably the most innovative part of the Langchain framework: **Langchain Expression Language (LCEL)**. In the demo we saw a very basic implementation with:

```python 
chain = prompt_template | model | parser
```

With LCEL, you can chain together anything that is a `Runnable` and by `Runnable` we mean anything that can use the `.invoke()` method. That includes:

* Chat models 
* Output parsers 
* Prompt Templates

All sequences (also called `RunnableSequence`) is either separated with a `|` or using the `.pipe()` method

In [8]:
# Create a environment variable
# %env MISTRAL_API_KEY=

# Now use echo to verify it's set
#!echo $MISTRAL_API_KEY

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI

system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

model = ChatMistralAI(model="mistral-large-latest")
parser = StrOutputParser()

pipe_sequence = (
    prompt_template
    .pipe(model)
    .pipe(parser)
)

pipe_sequence.invoke({
    "language": "French", 
    "text": "Jedha was the Jedis' home planet"
})

'The translation of "Jedha was the Jedis\' home planet" into French is:\n\n"Jedha était la planète d\'origine des Jedi."\n\nHere\'s a breakdown:\n- Jedha = Jedha\n- was = était\n- the = la\n- Jedis\' = des Jedi (Note that "Jedi" is invariable in French and doesn\'t take an apostrophe-s to show possession.)\n- home planet = planète d\'origine'

In [10]:
print(pipe_sequence.invoke({"language": "French","text": "Jedha was the Jedis' home planet"}))

The translation of "Jedha was the Jedis' home planet" into French is:

"Jedha était la planète d'origine des Jedi."

Here's a breakdown:
- Jedha = Jedha
- was = était
- the = la
- Jedis' = des Jedi (Note that "Jedi" is an invariable noun in French, so it doesn't take an apostrophe-s to show possession.)
- home planet = planète d'origine


### LangServe

The final piece for our application is **LangServe**. It is an additional layer on top of FastAPI that will:

* Create three sub-endpoints: `invoke`, `batch` and `stream` per endpoint 
* Create a playground at `/endpoint_your_setup/playground`
* Create a `/stream_log` endpoint to monitor intermediate steps from your chains which is great for debugging 
* Create a client SDK to easily call your endpoint

All you have to do is:

1. Create a Runnable
2. Wrap it around a `add_routes()` method like this:

```python 
from fastapi import FastAPI
from langserve import add_routes

app = FastAPI(
    title="Sample LangServe API",
    version="0.1",
    description="Simple FastAPI app that integrates LangServe"
)

add_routes(
    app, # this is your FastAPI instance
    pipe_sequence, # This is the Runnable chain that we defined above
    "/translate" # This is the endpoint 
)
```

<Note type="note">

You will need to repeat the `add_routes` for every endpoints you create for your application (Not very DRY I know).

</Note>

## Resources 📚📚

* [Build a simple LLM Application](https://python.langchain.com/docs/tutorials/llm_chain/)
* [La plateforme - Mistral](https://console.mistral.ai/)
* [All integrated ChatModels](https://python.langchain.com/docs/integrations/chat/)
* [LangServe](https://python.langchain.com/docs/langserve/)